In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display

# 네이트 뉴스 섹션 딕셔너리
section_dict = {'최신뉴스': '100', '정치': '200', '경제': '300', '사회': '400', '세계': '500', 'IT/과학': '600'}

def print_nate_news(section_name):
    sid = section_dict.get(section_name)
    if not sid:
        print(f"'{section_name}'은(는) 유효한 섹션 이름이 아닙니다.")
        return

    if section_name == '최신뉴스':
        url = f'https://news.nate.com/recent?mid=n0{sid}'
    else:
        url = f'https://news.nate.com/section?mid=n0{sid}'
        
    print(f'======> {url} {section_name} 뉴스 <======\n')

    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
    }

    res = requests.get(url, headers=req_header)

    if res.ok:
        html = res.text
        soup = BeautifulSoup(res.content, "html.parser")
        
        # [수정] '최신뉴스'와 다른 섹션의 스크래핑 로직을 분리
        if section_name == '최신뉴스':
            # '최신뉴스' 페이지 스크래핑 로직
            all_articles = soup.select("div.post-list-type1")
            if not all_articles:
                print("최신 뉴스 목록을 찾을 수 없습니다.")
                return

            for item in all_articles:
                title, link, img_src = "제목 없음", "#", None
                link_tag = item.select_one('dl > dt > a')
                if link_tag:
                    link = link_tag.get('href', '#')
                    if link.startswith('//'): link = 'https:' + link
                    
                    title_tag = link_tag.select_one('strong.tit')
                    if title_tag: title = title_tag.text.strip()

                img_tag = item.select_one('dd.thum > a > img')
                if img_tag and img_tag.has_attr('src'):
                    img_src = img_tag['src']
                    if img_src.startswith('//'): img_src = 'https:' + img_src
                
                # 출력
                if img_src: display(Image(url=img_src))
                print(f'📄 제목: {title}')
                print(f'🔗 링크: {link}\n')
                print('-' * 50)
                
        else:
            # 기존 '정치', '경제' 등 다른 섹션의 스크래핑 로직
            top_story = soup.select("div.mlt01")
            list_stories = soup.select("div.post_list li")
            all_articles = top_story + list_stories
            if not all_articles:
                print("뉴스 목록을 찾을 수 없습니다.")
                return

            for item in all_articles:
                title, link, img_src = "제목 없음", "#", None
                link_tag = item.find('a')
                if link_tag and link_tag.has_attr('href'):
                    link = link_tag['href']
                    if link.startswith('//'): link = 'https:' + link

                title_tag = item.select_one("strong.tit, strong.sub_tit")
                if title_tag: title = title_tag.text.strip()
                    
                img_tag = item.find('img')
                if img_tag and img_tag.has_attr('src'):
                    img_src = img_tag['src']
                    if img_src.startswith('//'): img_src = 'https:' + img_src
                
                # 출력
                if img_src: display(Image(url=img_src))
                print(f'📄 제목: {title}')
                print(f'🔗 링크: {link}\n')
                print('-' * 50)
            
    else:
        print(f'Error Code = {res.status_code}')

In [28]:
print_news('IT/과학')

======> https://news.nate.com/section?mid=n0600 IT/과학 뉴스 <======

